<a href="https://colab.research.google.com/github/hassanSattariNia/prediction_time_series/blob/main/Albert-v2-lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch peft scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [3]:
# تنظیمات LORA برای fine-tuning
lora_config = LoraConfig(
    r=16,               # تعداد rankها برای low-rank adaptation
    lora_alpha=32,      # ضریب مقیاس‌دهی
    target_modules=["query", "value"],  # ماژول‌هایی که LORA به آنها اعمال می‌شود
    lora_dropout=0.1,   # Dropout برای LORA
)

# بارگذاری مدل و توکنایزر
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# ادغام LORA با مدل
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# بارگذاری دیتاست GLUE (MRPC)
dataset = load_dataset("glue", "mrpc")
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Train set size: 3668
Validation set size: 408
Test set size: 1725


In [5]:
# تابع پیش‌پردازش برای توکنایز کردن دیتاست
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")

# توکنایز کردن دیتاست
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
# تعریف آرگومان‌های تمرین مدل
training_args = TrainingArguments(
    output_dir="./results",               # دایرکتوری برای ذخیره مدل‌ها و لاگ‌ها
    evaluation_strategy="epoch",          # استراتژی ارزیابی در پایان هر epoch
    per_device_train_batch_size=16,       # سایز batch برای train
    per_device_eval_batch_size=16,        # سایز batch برای validation
    num_train_epochs=5,                   # تعداد epochهای تمرین
    weight_decay=0.01,                    # میزان weight decay
    logging_dir="./logs",                 # دایرکتوری لاگ
    logging_steps=500,                    # فاصله زمانی برای لاگ کردن
    save_total_limit=2                    # تعداد محدود مدل‌هایی که ذخیره می‌شوند
)

# تعریف Trainer
trainer = Trainer(
    model=model,                          # مدل با LORA
    args=training_args,                   # تنظیمات تمرین
    train_dataset=tokenized_dataset['train'],  # دیتاست تمرین
    eval_dataset=tokenized_dataset['validation'], # دیتاست ارزیابی
    tokenizer=tokenizer,                  # توکنایزر برای پردازش
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# آموزش مدل
trainer.train()

# ارزیابی مدل
results = trainer.evaluate()
print("Evaluation Results:", results)

Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,0.592500,No log
4,0.592500,No log
5,0.522300,No log


Evaluation Results: {'eval_runtime': 16.2668, 'eval_samples_per_second': 25.082, 'eval_steps_per_second': 1.598, 'epoch': 5.0}


In [10]:
print(model)

PeftModel(
  (base_model): LoraModel(
    (model): AlbertForSequenceClassification(
      (albert): AlbertModel(
        (embeddings): AlbertEmbeddings(
          (word_embeddings): Embedding(30000, 128, padding_idx=0)
          (position_embeddings): Embedding(512, 128)
          (token_type_embeddings): Embedding(2, 128)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (encoder): AlbertTransformer(
          (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
          (albert_layer_groups): ModuleList(
            (0): AlbertLayerGroup(
              (albert_layers): ModuleList(
                (0): AlbertLayer(
                  (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                  (attention): AlbertAttention(
                    (query): lora.Linear(
                      (base_layer): Linear(in_features=768, out